The program produces an output for the NYC subway stations and the Airbnb locations, where frequency of criminal activities is the lowest and the highest in Manhattan.  
Three data sets used: 
    - NYC Subway Entrances / Stations on NYC map
    - NYPD Complain Locations on NYC map
    - Airbnb locations – filtered for a favorable rating location

There was several data scrubbing done in light of the limited capabilities to test the data (too long of processing time) for the output parameters. The data is restricted to 379 Manhattan Subway stations, while the Airbnb data was filtered to match Manhattan with the top 85% rated locations.The NYC Complaints from 2018 data, which had about 500K rows.

The final output has the top 10 subway stations with highest and lowest criminal on a NYC Manhattan map.  The final output will show the top 10 safest subway stations and all highly rated airbnbs within 200 meters of each of them. 


**As per the analysis these were the top ten safest subway stations in Manhatten (The entire code is below)**

In [0]:
safest['subway'] = names_safest
safest

,station,avg_lat,avg_lon,complaint,subway
170,Hudson Blvd East between 33rd St & 34th St,40.75493,-74.00095,3.0,34 Street - Hudson Yards Station
193,Greenwich St & Morris St,40.70649,-74.01414,8.0,Bowling Green Station
271,2nd Ave & 94th St,40.78294,-73.94798,11.0,96 Street Station - Q
85,Central Park West & 87th St,40.78612,-73.96901,11.0,86 Street Station
134,Central Park West & 81st St,40.78212,-73.97188,15.0,81 Street - Museum of Natural History Station
87,Central Park West & 70th St,40.77494,-73.97714,17.0,72 Street Station
83,Central Park West & 97th St,40.79244,-73.96437,17.0,"96 Street Station - B,C"
77,Saint Nicholas Ave & 137th St,40.81846,-73.94737,20.0,135 Street Station
111,6th Ave & 16th St,40.73869,-73.99587,23.0,14 Street Station
158,6th Ave & 49th St,40.75929,-73.98085,25.0,47-50 Streets Rockefeller Center Station


**And these were the top ten dangerous subway stations in Manhatten**

In [0]:
most_dangerous['subway']=names_dangerous
most_dangerous

,station,avg_lat,avg_lon,complaint,subway
119,Lexington Avenue & 125th Street,40.80441,-73.93723,736.0,"125 Street Station - 4,5,6"
68,Broadway & 14th St,40.73497,-73.99110,702.0,14 Street - Union Sq Station
226,Broadway & 43rd St,40.75648,-73.98631,611.0,Times Sq - 42 St Station
129,Saint Nicholas Ave & 125th St,40.81077,-73.95260,526.0,"125 Street Station - A,B,C,D"
23,Broadway & Prince St,40.72424,-73.99779,490.0,Prince Street Station
14,6th Ave & 34th St,40.74974,-73.98756,455.0,34 St - Herald Sq Station
191,Varick St & North Moore St,40.71990,-74.00677,454.0,Franklin Street Station
177,Broadway & 95th St,40.79447,-73.97205,427.0,"96 Street Station - 1,2,3"
248,Church St & Dey St,40.71097,-74.01070,411.0,Fulton Street Station
137,8th Ave & 58th St,40.76731,-73.98270,389.0,59 St - Columbus Circle



# Code

## The following code was used to clean up the New York Subway Station database. The steps that needed to be taken were:
- First, create a list to obtain a dataset with just the columns OBJECTID, NAME, the_geom, and Line. These columns refer to the following information: 
  - OBJECTID: ID number for each subway station and entrances 
  - NAME: The name of each subway station and entrance 
  - the_geom: the latitude and longitude of each subway station 
  - Line: the lines available at each subway station
- Next, I used regular expressions to remove unwanted text from the location data and then split the_geom column into seperate columns of latitude and longitude. 
- The data included several of the same subway stations' different entrances, so I processed the NAME column so that we could group the subway stations with the same name, regardless of the entrance.
- I then took the latitudes and longitudes of each subway station (at all entrances) and averaged them to determine a station longitude and latitude.


In [0]:
import pandas as pd
import numpy as np

In [0]:
from sqlalchemy import create_engine

import MySQLdb as mdb
import re
# The connect() method has four parameters. 
# The first parameter is the host, where the MySQL database is located. 
# In our case it is a localhost, e.g. our computer. 
# The second parameter is the database user name. 
# It is followed by the user's account password. 
# The final parameter is the database name.
host = 'bigdata.stern.nyu.edu'
username = 'dealingF19GB1'
password = 'dealingF19GB1!!'
database = 'dealingF19GB1'

con = mdb.connect(host, username, password, database, 
                charset='utf8', use_unicode=True);
cur = con.cursor()
cur.execute("select * from subway_entrances")
listlatlon = list()



for i in range(cur.rowcount):

    row = cur.fetchone()
    listlat = list()
    
    listlat.append(row[0])
    listlat.append(row[2])
    listlat.append(row[3])
    listlat.append(row[4])
    listlatlon.append(listlat)
print(listlatlon[0:10])


[[1, '15th St & Kings Hwy at SE corner', 'POINT (-73.95776900005806 40.6088350009905)', 'B-Q'], [2, '15th St & Avenue U at SE corner', 'POINT (-73.95582799955477 40.59899100093364)', 'B-Q'], [3, '15th St & Gravesend Neck Rd at NE corner', 'POINT (-73.9551200002249 40.595020000759654)', 'B-Q'], [4, '15th St & Sheepshead Bay Rd at SE corner', 'POINT (-73.954223999936 40.58730200083184)', 'B-Q'], [5, None, 'POINT (-73.96025800029547 40.57798600098538)', 'B-Q'], [6, 'Branch 7 St & Brighton Beach Rd at NE corner', 'POINT (-73.96052400057089 40.577931001322625)', 'B-Q'], [7, 'Branch 7 St & Brighton Beach Rd at SE corner', 'POINT (-73.96023100018317 40.577746001029006)', 'B-Q'], [8, 'Branch 7 St & Brighton Beach Rd at SE corner', 'POINT (-73.96043800015305 40.577697001130275)', 'B-Q'], [9, 'Ocean Pkwy & Brighton Beach Rd at NE corner', 'POINT (-73.96789400057325 40.57642300107101)', 'Q'], [10, 'Ocean Pkwy & Brighton Beach Rd at NE corner', 'POINT (-73.96759099960158 40.576445000451336)', 'Q']

In [0]:
entrance_needed =  pd.DataFrame(listlatlon,columns=['OBJECTID' , 'NAME', 'the_geom', 'LINE'])
entrance_needed.head(10)

,OBJECTID,NAME,the_geom,LINE
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q
4,5,None,POINT (-73.96025800029547 40.57798600098538),B-Q
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q


In [0]:
entrance_needed['LatandLong'] = entrance_needed['the_geom'].str.extract(r"\((.*?)\)", expand=False)
entrance_needed.head(10)

,OBJECTID,NAME,the_geom,LINE,LatandLong
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95776900005806 40.6088350009905
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95582799955477 40.59899100093364
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.9551200002249 40.595020000759654
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.954223999936 40.58730200083184
4,5,None,POINT (-73.96025800029547 40.57798600098538),B-Q,-73.96025800029547 40.57798600098538
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052400057089 40.577931001322625
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q,-73.96023100018317 40.577746001029006
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q,-73.96043800015305 40.577697001130275
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q,-73.96789400057325 40.57642300107101
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q,-73.96759099960158 40.576445000451336


In [0]:
entrance_needed['LatandLong'] = entrance_needed['the_geom'].str.extract(r"\((.*?)\)", expand=False)
entrance_needed.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,LatandLong
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,-73.95776900005806 40.6088350009905
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,-73.95582799955477 40.59899100093364
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,-73.9551200002249 40.595020000759654
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,-73.954223999936 40.58730200083184
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793,-73.96052400057089 40.577931001322625
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q,-73.96023,40.57775,-73.96023100018317 40.577746001029006
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q,-73.96044,40.57770,-73.96043800015305 40.577697001130275
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q,-73.96789,40.57642,-73.96789400057325 40.57642300107101
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q,-73.96759,40.57645,-73.96759099960158 40.576445000451336
10,11,Ocean Pkwy & Brighton Beach Rd at SE corner,POINT (-73.96787500037775 40.576209000557355),Q,-73.96788,40.57621,-73.96787500037775 40.576209000557355


In [0]:
entrance_needed[['Longitude','Latitude']] = entrance_needed.LatandLong.str.split(" ",expand=True,)
entrance_needed.head(10)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,LatandLong
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95776900005806,40.6088350009905,-73.95776900005806 40.6088350009905
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95582799955477,40.59899100093364,-73.95582799955477 40.59899100093364
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.9551200002249,40.595020000759654,-73.9551200002249 40.595020000759654
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.954223999936,40.58730200083184,-73.954223999936 40.58730200083184
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052400057089,40.577931001322625,-73.96052400057089 40.577931001322625
6,7,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96023100018317 40.577746001029006),B-Q,-73.96023100018317,40.577746001029006,-73.96023100018317 40.577746001029006
7,8,Branch 7 St & Brighton Beach Rd at SE corner,POINT (-73.96043800015305 40.577697001130275),B-Q,-73.96043800015305,40.577697001130275,-73.96043800015305 40.577697001130275
8,9,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96789400057325 40.57642300107101),Q,-73.96789400057325,40.57642300107101,-73.96789400057325 40.57642300107101
9,10,Ocean Pkwy & Brighton Beach Rd at NE corner,POINT (-73.96759099960158 40.576445000451336),Q,-73.96759099960158,40.576445000451336,-73.96759099960158 40.576445000451336
10,11,Ocean Pkwy & Brighton Beach Rd at SE corner,POINT (-73.96787500037775 40.576209000557355),Q,-73.96787500037775,40.576209000557355,-73.96787500037775 40.576209000557355


In [0]:
entrance_needed['Latitude'] = entrance_needed['Latitude'].astype(float).round(5)
entrance_needed['Longitude'] = entrance_needed['Longitude'].astype(float).round(5)
entrance_needed = entrance_needed.drop(columns=['LatandLong'])
entrance_needed

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793
...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292


In [0]:
entrance_needed = entrance_needed.dropna(axis=0, subset=['NAME'])
entrance_needed

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793
...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292


In [0]:
entrance_needed['station'] = entrance_needed['NAME'].str.extract(r'(.*)at')
entrance_needed

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,15th St & Kings Hwy
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,15th St & Avenue U
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,15th St & Gravesend Neck Rd
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,15th St & Sheepshead Bay Rd
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793,Branch 7 St & Brighton Beach Rd
...,...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,NaN
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410,2nd Ave & 96th St
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392,2nd Ave & 95th St
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St


In [0]:
entrance_needed['entrance'] = entrance_needed['NAME'].str.extract(r'at(.*)')
entrance_needed

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance
0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,15th St & Kings Hwy,SE corner
1,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,15th St & Avenue U,SE corner
2,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,15th St & Gravesend Neck Rd,NE corner
3,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,15th St & Sheepshead Bay Rd,SE corner
5,6,Branch 7 St & Brighton Beach Rd at NE corner,POINT (-73.96052400057089 40.577931001322625),B-Q,-73.96052,40.57793,Branch 7 St & Brighton Beach Rd,NE corner
...,...,...,...,...,...,...,...,...
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,NaN,NaN
1924,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410,2nd Ave & 96th St,SW corner
1925,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392,2nd Ave & 95th St,NW corner
1926,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St,NE corner


In [0]:
index_wo_at = entrance_needed.index[(entrance_needed['NAME'].str.find(r'at')==-1)].tolist()
entrance_needed.loc[index_wo_at, 'station'] = entrance_needed.loc[index_wo_at, 'NAME']
entrance_needed[(entrance_needed['NAME'].str.find(r'at')==-1)]

,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance
79,80,Norwood,POINT (-73.8789840000952 40.68164300107813),J-Z,-73.87898,40.68164,Norwood,NaN
81,82,Cleveland,POINT (-73.88571600056578 40.6795300007475),J,-73.88572,40.67953,Cleveland,NaN
83,84,Van Siclen,POINT (-73.89156599958824 40.6778640010583),J-Z,-73.89157,40.67786,Van Siclen,NaN
778,779,Church St,POINT (-74.0101649998177 40.71179300109741),A-C-E-2-3,-74.01016,40.71179,Church St,NaN
779,780,Church St,POINT (-74.01020399993797 40.71171000117648),A-C-E-2-3,-74.01020,40.71171,Church St,NaN
891,892,Pitkin,POINT (-73.87187499969077 40.67555400055863),A-C,-73.87187,40.67555,Pitkin,NaN
1081,1082,Hudson Blvd East between 33rd St & 34th St (no...,POINT (-74.00092352989824 40.75497493443128),7,-74.00092,40.75497,Hudson Blvd East between 33rd St & 34th St (no...,NaN
1082,1083,Hudson Blvd East between 33rd St & 34th St (so...,POINT (-74.00098158502907 40.75489825502214),7,-74.00098,40.75490,Hudson Blvd East between 33rd St & 34th St (so...,NaN
1486,1487,7 BENTLEY STREET,POINT (-74.25283251705811 40.51211370971486),SIR,-74.25283,40.51211,7 BENTLEY STREET,NaN
1923,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,2nd Ave & bet 83rd & 84th St,NaN


In [0]:
subway_data = pd.read_csv('entrance_needed.csv')
subway_data

,Unnamed: 0,OBJECTID,NAME,the_geom,LINE,Longitude,Latitude,station,entrance,avg_lon,avg_lat
0,0,1,15th St & Kings Hwy at SE corner,POINT (-73.95776900005806 40.6088350009905),B-Q,-73.95777,40.60884,15th St & Kings Hwy,SE corner,-73.957790,40.608950
1,1,282,15th St & Kings Hwy at NE corner,POINT (-73.95780799949603 40.609062000781535),B-Q,-73.95781,40.60906,15th St & Kings Hwy,NE corner,-73.957790,40.608950
2,2,2,15th St & Avenue U at SE corner,POINT (-73.95582799955477 40.59899100093364),B-Q,-73.95583,40.59899,15th St & Avenue U,SE corner,-73.955830,40.598990
3,3,3,15th St & Gravesend Neck Rd at NE corner,POINT (-73.9551200002249 40.595020000759654),B-Q,-73.95512,40.59502,15th St & Gravesend Neck Rd,NE corner,-73.955120,40.595020
4,4,4,15th St & Sheepshead Bay Rd at SE corner,POINT (-73.954223999936 40.58730200083184),B-Q,-73.95422,40.58730,15th St & Sheepshead Bay Rd,SE corner,-73.954220,40.587300
...,...,...,...,...,...,...,...,...,...,...,...
1894,1894,1928,2nd Ave & bet 83rd & 84th St,POINT (-73.95281423699555 40.77618770946792),Q,-73.95281,40.77619,2nd Ave & bet 83rd & 84th St,itself,-73.952810,40.776190
1895,1895,1929,2nd Ave & 96th St at SW corner,POINT (-73.94747299691045 40.784098884955554),Q,-73.94747,40.78410,2nd Ave & 96th St,SW corner,-73.947470,40.784100
1896,1896,1930,2nd Ave & 95th St at NW corner,POINT (-73.94761889814153 40.783916088309326),Q,-73.94762,40.78392,2nd Ave & 95th St,NW corner,-73.947620,40.783920
1897,1897,1931,2nd Ave & 94th St at NE corner,POINT (-73.94772938539093 40.78292438898596),Q,-73.94773,40.78292,2nd Ave & 94th St,NE corner,-73.947985,40.782945


In [0]:
north = 40.87700
south = 40.70000
west = -74.03000
east = -73.90000
subway_needed = subway_data[((subway_data['Latitude']>south) & (subway_data['Latitude']<north))]
subway_needed = subway_needed[((subway_data['Longitude']>west) & (subway_data['Longitude']<east))]
subway_needed['avg_lat'] = subway_needed['avg_lat'].round(5)
subway_needed['avg_lon'] = subway_needed['avg_lon'].round(5)
subway_needed = subway_needed[['station','avg_lat','avg_lon']]
subway_needed.drop_duplicates(inplace=True)
subway_needed

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,station,avg_lat,avg_lon
96,Flushing Ave & Broadway,40.70070,-73.94196
99,Moore St & Broadway,40.70434,-73.94827
101,Hooper St & Broadway,40.70718,-73.95435
103,Marcy Ave & Broadway,40.70841,-73.95792
109,Norfolk St & Delancey St,40.71848,-73.98742
...,...,...,...
1891,2nd Ave & 86th St,40.77781,-73.95148
1894,2nd Ave & bet 83rd & 84th St,40.77619,-73.95281
1895,2nd Ave & 96th St,40.78410,-73.94747
1896,2nd Ave & 95th St,40.78392,-73.94762


In [0]:
subway_needed.to_csv('subway_needed_manhattan.csv',index=False)

## The following code was used to clean up the NYC Complaint database. The steps that needed to be taken were:

- First, create a table (labeled: complaint_needed) to obtain a dataset with just the columns CMPNT_NUM, Latitude, Longitude, BORO_NM, LAW_CAT_CD, OFNS_DESC, CMPLNT_FR_DT, CMPLNT_FR_TM. These columns refer to the following information: 
  - CMPNT_NUM: The complaint number
  - Latitude: The latitude of the complaint 
  - Longitude: The longitude of the complaint
  - BORO_NM: The boroughs the complaint occured in
  - LAW_CAT_CD: The category of the specific complaint (Violation, Midsdemeanor, of Felony)
  - OFNS_DESC: A description of the offense the complaint was made about
  - CMPLNT_FR_DT: The date of the complaint 
  - CMPLNT_FR_TM: The time of the complaint

- Any complaints that did not have location (latitude and longitude) data were dropped out of the dataset
-  Restricted the data to 2018.


In [0]:
import pandas as pd
import numpy as np

In [0]:
complaintdata = pd.read_csv('NYC_ComplainData.csv')
# complaintdata.head()

In [0]:
# complaintdata.columns
complaint_needed = complaintdata[['CMPLNT_NUM', 'Latitude', 'Longitude', 'BORO_NM', 'LAW_CAT_CD', 'OFNS_DESC', 'CMPLNT_FR_DT', 'CMPLNT_FR_TM']]
complaint_needed

In [0]:
complaint_needed['CMPLNT_FR_DT'] = pd.to_datetime(complaint_needed['CMPLNT_FR_DT'], format='%m/%d/%Y', errors='coerce')

In [0]:
complaint_needed = complaint_needed.dropna(axis=0, subset=['Latitude', 'Longitude'])
complaint_needed

In [0]:
start_date = '2018-01-01'
mask = (complaint_needed['CMPLNT_FR_DT'] > start_date)
complaint_needed = complaint_needed.loc[mask]
complaint_needed

In [0]:
complaint_needed['Latitude'] = complaint_needed['Latitude'].round(5)
complaint_needed['Longitude'] = complaint_needed['Longitude'].round(5)
complaint_needed

In [0]:
complaint_needed.to_csv('complaint_needed.csv')

In [0]:
complaint_needed = pd.merge(complaint_needed, complaint_count, on=['Latitude','Longitude'])
complaint_needed.drop_duplicates(inplace=True)
complaint_needed

,Latitude,Longitude,count
0,40.75043,-73.98928,1238
1238,40.75664,-73.98837,298
1536,40.72543,-73.99208,30
1566,40.80277,-73.93860,12
1578,40.76187,-73.96636,255
...,...,...,...
111303,40.77238,-73.96079,1
111304,40.75931,-73.96220,1
111305,40.71040,-73.99518,1
111306,40.82665,-73.94306,1


In [0]:
complaint_needed.to_csv('complaint_needed_manhattan.csv',index=False)


- Count the number of crimes (complaints) that occured within 100 meters of each subway station
- Then,  select the 10 stations with the highest occurance of complaints nearby to be the most dangerous and the 10 stations with the lowest occurence of complaints nearby to be the safest.
- Finally,  plot both of these top 10 lists on a map to display visually where these locations are in Manhattan. 

In [0]:
import pandas as pd
import numpy as np

In [0]:
complaint_data = pd.read_csv('complaint_needed_manhattan.csv')
complaint_data

,Latitude,Longitude,count
0,40.75043,-73.98928,1238
1,40.75664,-73.98837,298
2,40.72543,-73.99208,30
3,40.80277,-73.93860,12
4,40.76187,-73.96636,255
...,...,...,...
8829,40.77238,-73.96079,1
8830,40.75931,-73.96220,1
8831,40.71040,-73.99518,1
8832,40.82665,-73.94306,1


In [0]:
subway_data = pd.read_csv('subway_needed_manhattan.csv')
subway_data

,station,avg_lat,avg_lon
0,Flushing Ave & Broadway,40.70070,-73.94196
1,Moore St & Broadway,40.70434,-73.94827
2,Hooper St & Broadway,40.70718,-73.95435
3,Marcy Ave & Broadway,40.70841,-73.95792
4,Norfolk St & Delancey St,40.71848,-73.98742
...,...,...,...
374,2nd Ave & 86th St,40.77781,-73.95148
375,2nd Ave & bet 83rd & 84th St,40.77619,-73.95281
376,2nd Ave & 96th St,40.78410,-73.94747
377,2nd Ave & 95th St,40.78392,-73.94762


### In order to determine the distance between subway station and complaint locations,  Geopy package was used to calculate the distance between two locations by latitude and longitude.

In [0]:
from geopy.distance import geodesic
def geodistance(location1, location2):
    distance = geodesic(location1,location2).m
    return distance

# Below is a visualization of the top 10 most dangerous subway stations (determined by highest number of complaints within 100m) on the map

In [0]:
import folium
fmap = folium.Map(location=[40.73061, -73.93524], zoom_start=12, tiles = 'cartodbpositron')
# fmap
for name, row in most_dangerous.iterrows():
    size = row['complaint']/50
    color = 'red'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)
    
fmap

# Below is a visualization of the top 10 safest subway stations (determined by lowest number of complaints within 100m) on the map

In [0]:
import folium
fmap = folium.Map(location=[40.73061, -73.93524], zoom_start=12, tiles = 'cartodbpositron')
# fmap
for name, row in safest.iterrows():
    size = 4
    color = 'blue'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)
    
fmap

## put top 10 dangerous and top 10 safest subway stations on the same map

In [0]:
import folium
fmap = folium.Map(location=[40.73061, -73.93524], zoom_start=12, tiles = 'cartodbpositron')
# fmap
for name, row in most_dangerous.iterrows():
    size = row['complaint']/50
    color = 'red'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)

for name, row in safest.iterrows():
    size = 4
    color = 'blue'
    
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Subway: </strong>" + str(row["subway"]) + \
           "<br><strong>Latitude: </strong>" + str(row["avg_lat"]) + \
           "<br><strong>Longitude: </strong>" + str(row["avg_lon"]) + \
           "<br><strong>Complaint: </strong>" + str(row["complaint"])
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)
    
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        popup = popup,
                        fill_color = color,
                       ).add_to(fmap)
    
fmap

## The following code was used to clean up the New York Airbnb database. The steps that needed to be taken were:
- First, create a list to obtain a dataset with just the columns id	name, neighbourhood_group_cleansed, neighbourhood_cleansed	neighbourhood, latitude, longitude, review_scores_rating, and number_of_reviews. These columns refer to the following information: 
  - id name: ID number for each Airbnb 
  - neighbourhood_group_cleansed: The borough location of each airbnb
  - neighbourhood_cleansed: The neighborhood location of each airbnb
  - neighbourhood: Additional neighborhood information of each airbnb
  - latitude: The latitude of each Airbnb 
  - longitude: The longitude of each Airbnb
  - review_scores_rating: The average review rating of each Airbnb 
  - number_of_reviews: The amount of reviews each Airbnb has recieved 
- Then, the data was filtered just show the best Airbnb locations based on ratings and number of reviews.
- Similarly to the complaint and subway location datasets, the data was limited to Manhattan locations. 


In [0]:
import pandas as pd
import numpy as np


### Based on the previous analysis of the safest subway stations in Manhattan, it is determined what Airbnb locations are accesible to these low-crime transit areas. Based on the list of the top 10 safest subway stations, Airbnb was determined within 200 meters of each station. With this tool, visitors to NYC will be able to determine the best Airbnb options that are near the safest subway stations in the city!

In [0]:
safest_station = pd.read_csv('safest.csv')

In [0]:

safest_station



,station,avg_lat,avg_lon,complaint
0,Hudson Blvd East between 33rd St & 34th St,40.75493,-74.00095,3.0
1,Greenwich St & Morris St,40.70649,-74.01414,8.0
2,2nd Ave & 94th St,40.78294,-73.94798,11.0
3,Central Park West & 87th St,40.78612,-73.96901,11.0
4,Central Park West & 81st St,40.78212,-73.97188,15.0
5,Central Park West & 70th St,40.77494,-73.97714,17.0
6,Central Park West & 97th St,40.79244,-73.96437,17.0
7,Saint Nicholas Ave & 137th St,40.81846,-73.94737,20.0
8,West Broadway & Franklin St,40.71900,-74.00668,21.0
9,6th Ave & 16th St,40.73869,-73.99587,23.0


In [0]:
from geopy.distance import geodesic
def geodistance(location1, location2):
    distance = geodesic(location1,location2).m
    return distance

In [0]:
import time

count = 0

for id1,row1 in safest_station.iterrows():
    number_of_airbnb = 0
    location1 = (row1['avg_lat'], row1['avg_lon'])
    
    for id2,row2 in top_rating_bnb.iterrows():
        location2 = (row2['latitude'], row2['longitude'])
        if(geodistance(location1,location2) <=200):
            number_of_airbnb = number_of_airbnb + 1
            neighbor = row2['neighbourhood_cleansed']
            
    safest_station.loc[id1,'num_bnb'] = number_of_airbnb
    safest_station.loc[id1,'neighbor'] = neighbor
    #count = count + 1
safest_station


,station,avg_lat,avg_lon,complaint,num_bnb,neighbor
0,Hudson Blvd East between 33rd St & 34th St,40.75493,-74.00095,3.0,2.0,Hell's Kitchen
1,Greenwich St & Morris St,40.70649,-74.01414,8.0,22.0,Financial District
2,2nd Ave & 94th St,40.78294,-73.94798,11.0,23.0,Upper East Side
3,Central Park West & 87th St,40.78612,-73.96901,11.0,9.0,Upper West Side
4,Central Park West & 81st St,40.78212,-73.97188,15.0,7.0,Upper West Side
5,Central Park West & 70th St,40.77494,-73.97714,17.0,11.0,Upper West Side
6,Central Park West & 97th St,40.79244,-73.96437,17.0,6.0,Upper West Side
7,Saint Nicholas Ave & 137th St,40.81846,-73.94737,20.0,28.0,Harlem
8,West Broadway & Franklin St,40.71900,-74.00668,21.0,10.0,Tribeca
9,6th Ave & 16th St,40.73869,-73.99587,23.0,10.0,Chelsea


In [0]:
dfsafeairbnbs = pd.DataFrame()
for id1,row1 in safest_station.iterrows():
   
    location1 = (row1['avg_lat'], row1['avg_lon'])
   
    for id2,row2 in top_rating_bnb.iterrows():
        location2 = (row2['latitude'], row2['longitude'])
        if(geodistance(location1,location2) <=200):
            
            dfsafeairbnbs=dfsafeairbnbs.append(row2, ignore_index=True)
#             dfsafeairbnbs.append({'Airbnbname':row2['name']},ignore_index=True)
#             dfsafeairbnbs.append({'latitude':row2['latitude']},ignore_index=True)
#             dfsafeairbnbs.append({'longitude':row2['longitude']},ignore_index=True)
            
            

dfsafeairbnbs



,Unnamed: 0,id,latitude,longitude,name,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,number_of_reviews,review_scores_rating
0,5838.0,4472978.0,40.75531,-73.99945,Charming Studio in a Great location,Manhattan,Hell's Kitchen,Manhattan,141.0,99.0
1,19104.0,16014791.0,40.75536,-73.99950,Clean style apt. Central midtwn near Javits,Midtown,Hell's Kitchen,Manhattan,192.0,90.0
2,38822.0,32240966.0,40.70620,-74.01192,Sonder | Stock Exchange | Sleek 3BR + Lounge,Financial District,Financial District,Manhattan,14.0,100.0
3,39811.0,33072230.0,40.70645,-74.01186,Sonder | Stock Exchange | Sleek 3BR + Laundry,Financial District,Financial District,Manhattan,11.0,100.0
4,24384.0,20518456.0,40.70744,-74.01521,Bright & Luxurious Studio in Financial District,Manhattan,Financial District,Manhattan,43.0,99.0
...,...,...,...,...,...,...,...,...,...,...
123,1951.0,918049.0,40.73949,-73.99483,Union Sq European Serene apt,Manhattan,Chelsea,Manhattan,48.0,96.0
124,32204.0,26934498.0,40.73968,-73.99746,Charming room in Chelsea,Chelsea,Chelsea,Manhattan,45.0,92.0
125,4198.0,3013158.0,40.73893,-73.99464,Alcove Studio in Downtown Manhattan,Manhattan,Chelsea,Manhattan,12.0,92.0
126,1876.0,858223.0,40.73818,-73.99770,Entire flat west village,Chelsea,Chelsea,Manhattan,86.0,91.0


In [0]:
import folium
airbnbmap = folium.Map(location=[40.73, -74], zoom_start=13, tiles='cartodbpositron')
airbnbmap

# Once again, below is a visualization of the top 10 safest subway stations (determined by lowest number of complaints within 100m) on the map 

In [0]:
for name, row in safest_station.iterrows():
    opacity =  1.0
    color = "Blue"
    size = 8
    
      # The code below defines a pop-up for each station with details such as 
    # the address, number of bikes, capacity, etc.
    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Station Name: </strong>" + row["station"] + \
           "<br><strong>Number of Complaint: </strong>" + str(row["complaint"] )
        
        
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)

  
    # We create a marker on the map and we add it to the map
    folium.CircleMarker(location=[row["avg_lat"], row["avg_lon"]], 
                        radius = size,
                        fill_opacity = opacity,
                        fill_color = color,
                         popup = popup,
                       ).add_to(airbnbmap)
    
airbnbmap



# And finally, below is a visualization of the best Airbnb options located within 200 meters of the top 10 safest subway stations in Manhattan. The visitors can click on the map to see the popup information about the airbnb locations and choose where they want to live. 

In [0]:
safeairbnbnames = list()
for name, row in dfsafeairbnbs.iterrows():
    color = "red"
    size = 1

    html = "<p style='font-family:sans-serif;font-size:11px'>" + \
           "<strong>Airbnb Name: </strong>" + row["name"] + \
           "<br><strong>Rating Scores: </strong>" + str(row["review_scores_rating"] ) + \
           "<br><strong>Number of review: </strong>" + str(row["number_of_reviews"])
       
    iframe = folium.IFrame(html=html, width=200, height=60)
    popup = folium.Popup(iframe, max_width=200)

    
    
    # We create a marker on the map and we add it to the map
    folium.Marker(location=[row["latitude"], row["longitude"]], 
                        radius = size,
                        popup = popup, 
                        #fill_opacity = opacity,
                        fill_color = color,
                        icon=folium.Icon(icon='cloud',color='red')
                       ).add_to(airbnbmap)
   
    
airbnbmap
